In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
from tqdm import tqdm
import matplotlib.pyplot as plt
from lmoments3 import distr

from modules.api import load_config, extract_regions, extract_coloc 
from modules.api import corrmatrix, samlmomgev, Simulation

In [2]:
# Load parameters from config file
config = load_config(Path('config.json'))
dir_root = config['dir_root']
region_file = config['region_file']
prism_file = config['prism_file']
meta_file = config['meta_file']
coloc_file = config['coloc_file']
ARI = np.array(config['ARI']) # Average Recurrence Interval
aep = 1 - 1/ARI  # Annual Exceedance Probability
base_durations = config['base_durations']
prism_durations = config['prism_durations']
nsim = config['nsim'] # Number of Monte Carlo Simulations
boundprob = config['boundprob'] # 90% CI
nmom = config['nmom'] # number of L-moments

distributions = [distr.gev, distr.gno, distr.glo, distr.pe3] 

In [3]:
# Read metdata
df_meta = pd.read_parquet(Path(meta_file), engine='pyarrow')
# Read PRISM MAM
prism_mam_df = pd.read_csv(Path(prism_file), sep = '\s+')
prism_mam_df = prism_mam_df.rename(columns={'Site_ID': 'HDSC'}).set_index('HDSC', drop=True)
# Read regionalization information
sids, regions = extract_regions(Path(region_file))
coloc = extract_coloc(coloc_file)
# exclude station ids that are in buffer region.
#sids = coloc[(coloc['HDSC_ID'].isin(sids)) & (coloc['ST'].isin(['ID', 'WY', 'MT']))]['HDSC_ID'].unique()
df_meta1 = pd.concat([df_meta[df_meta['HDSC'] == sid] for sid in sids])
lat = df_meta1['LAT'].values
lon = df_meta1['LON'].values

In [4]:
quant = np.full((len(base_durations), len(sids), len(ARI)), np.nan)
lbound = np.full((len(base_durations), len(sids), len(ARI)), np.nan)
ubound = np.full((len(base_durations), len(sids), len(ARI)), np.nan)

In [5]:
np.random.seed(123)
for index1, (base, prism_duration) in enumerate(zip(base_durations, prism_durations)):
    print(f'Base Duration: {base}')
    # Read the AMS for specifc base duration
    ams_df = pd.read_parquet(Path(dir_root, f'df_ams_{base}.parquet.snappy'), engine='pyarrow')
    ams_df = ams_df.join(df_meta['HDSC'])
    ams_df = ams_df.set_index('HDSC', drop=True)
    for index2, sid in tqdm(enumerate(sids), total=len(sids), desc="Processing stations"):
        sid_region = [sid] + regions[sid]
        # Extract AMS for all stations in a region
        ams = ams_df[ams_df.index.isin(sid_region)].copy()
        # Get PRISM MAM
        mam_prism = prism_mam_df[prism_mam_df.index.isin(sid_region)][prism_duration].copy()
        # Calculate the at-site L-moments and GEV paameters
        lmoms, paras = samlmomgev(ams, mam_prism)

        # Get station MAM
        smam = mam_prism.loc[sid]
        if np.isnan(smam):
            if sid not in lmoms.index:
                #print(f'No MAM for {sid} at {base} duration. Proceed to the next station.')
                continue
            else:
                smam = lmoms.loc[sid]['mean']

        # Calculate inter-site correleation coefficient
        rho, sd_rho = corrmatrix(ams)
        # Run Monte Carlo Simulation
        sim = Simulation(sid, base, rho, sd_rho, ARI, lmoms, smam, nsim, distributions, boundprob)
        sim.run_simulations()
        quant[index1, index2, :], lbound[index1, index2, :], ubound[index1, index2, :] = sim.bounds()

Base Duration: 60m


Processing stations: 100%|██████████| 1521/1521 [29:08<00:00,  1.15s/it]


Base Duration: 06h


Processing stations: 100%|██████████| 1521/1521 [28:59<00:00,  1.14s/it]


Base Duration: 24h


...

Processing stations: 100%|██████████| 1521/1521 [48:25<00:00,  1.91s/it]


Base Duration: 30d


Processing stations: 100%|██████████| 1521/1521 [48:28<00:00,  1.91s/it]


Base Duration: 60d


Processing stations: 100%|██████████| 1521/1521 [48:27<00:00,  1.91s/it]


In [6]:
# Save data to netcdf file
print('Save data to netcdf file')

quant_da = xr.DataArray(quant, dims=['duration', 'hdsc', 'ari'],
                         coords={'duration': base_durations,
                                 'hdsc': sids, 'ari': ARI})

lbound_da = xr.DataArray(lbound, dims=['duration', 'hdsc', 'ari'], 
                         coords={'duration': base_durations,
                                 'hdsc': sids, 'ari': ARI})

ubound_da = xr.DataArray(ubound, dims=['duration', 'hdsc', 'ari'], 
                           coords={'duration': base_durations,
                                   'hdsc': sids, 'ari': ARI})

Save data to netcdf file


In [7]:
ds = xr.Dataset({
    'quantile': quant_da,
    'lbound': lbound_da,
    'ubound': ubound_da,
})

metadata = {
    'lbound': {
        'long_name': 'Lower Confidence Interval',
        'description': 'Lower confidence interval of the GEV estimate',
        'units': 'inches'
    },
    'ubound': {
        'long_name': 'Upper Confidence Interval',
        'description': 'Upper confidence interval of the GEV estimate',
        'units': 'inches'
    },
    'quantile': {
        'long_name': 'GEV Estimate',
        'description': 'Regional GEV Estimate of the annual maximum precipitation',
        'units': 'inches'
    }
}


for var_name, attrs in metadata.items():
    ds[var_name].attrs.update(attrs)

ds.coords['lat'] = ('hdsc', lat)
ds.coords['lon'] = ('hdsc', lon)

# ds.to_netcdf('regional_stats_montecarlo_cibounds.nc')
ds.to_netcdf('MonteCarlo_CIbounds.nc')